<a href="https://colab.research.google.com/github/Rajeevku1/AIAgentSamples/blob/main/MeetingMinutesAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata


In [2]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

# STEP 1: Transcribe Audio

In [ ]:
# Sign in to OpenAI using Secrets in Colab

AUDIO_MODEL = "gpt-4o-mini-transcribe"
audio_filename = "/content/meetingfile1.mp4"
# Open the file
audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
#print(transcription)

In [5]:
system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a RCLI Demo meeting.
Please write minutes in markdown, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [6]:
response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages, temperature=0.7)
transcription = response.choices[0].message.content
print("\n--- OpenAI Meeting Summary ---")


--- OpenAI Meeting Summary ---


In [7]:
print("\n\n")
display(Markdown(transcription))

# Minutes of RCLI Demo Meeting

**Date:** October 14, 2023  
**Location:** City Council Chambers, City Hall, 301 West 2nd Street, Austin, Texas  
**Attendees:** Members of the City of Austin City Council Audit and Finance Committee  

## Summary
The meeting was called to order at 1:00 PM with a quorum present. The primary agenda item was an executive session to discuss personnel matters related to the appointment of a new city auditor. The meeting adjourned at 3:06 PM after concluding the executive session.

## Discussion Points
- The committee convened to address personnel matters concerning the new city auditor appointment.
- No public speakers were signed up to address the committee.
- The meeting transitioned into executive session as per Texas Government Code section 551.074.

## Takeaways
- The appointment of a new city auditor is a priority for the committee.
- The executive session was necessary to ensure confidentiality regarding personnel discussions.

## Action Items
- **Action Item:** Complete the appointment process for the new city auditor.  
  **Owner:** City Council Audit and Finance Committee  
  **Due Date:** TBD (To be determined based on the outcome of the executive session)